### Load packages and churn data

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook
import googlemaps
import json

# display all columns of df
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = 80

In [2]:
churnDataOrig = pd.read_csv('churnData.csv', sep=';', engine='python', encoding='UTF-8')

### Have a first look at data

In [3]:
churnDataOrig.head()

,car_nr,ref_dt,target_event,NUM_CONSEC_SERVICES,SUM_INVOICE_AMOUNT_IN_SERVICE,SUM_INVOICE_AMOUNT,NUM_EVENTS,FIRST_EVT,LAST_EVT,LAST_MILEAGE,MEAN_MILEAGE_PER_MNTH,AVG_DIFF_MNTH,age_mnth,KANTON_LICENCE_PLATE,INSPECTION_INTERVAL_UID,LIST_PRICE,END_CUSTOMER_STATUS_UID,ACCIDENT_CAR,CAR_BRAND_UID,DESCRIPTION_DE,DESCRIPTION_DE2,FUEL_TYPE_UID,GEAR_TYPE_UID,WHEEL_DRIVE_UID,NUMBER_OF_DOORS_UID,GEAR_COUNT,EXTERIOR_COLOR_DE,INTERIOR_COLOR_DE,BASE_MARGIN,BUDGET_TYPE_IMPORT,SALES_TYPE,SALES_TYPE_IMPORT,CAR_OWNER_JOIN_NR,FUNCTION_TYPE_UID,PERSON_LANGUAGE_UID,TITLE,POSITION,EVT_BIRTH,PERSON_STREET,PERSON_STATE,PERSON_ZIP_CODE,PERSON_CITY,PERSON_ADRESS_COUNT,ownerAge,PARTNER_COMPANY_NR,NAME,ACTIVE,REGION_UID,SECTOR_UID,PARTNER_LANGUAGE_UID,IS_RETAIL_PARTNER,IS_PREFERRED_PARTNER,IS_DEALER,IS_SERVICE_PARTNER,RATING_UID,PARTNER_ADRESS_TYPE_UID,PARTNER_STREET,PARTNER_STATE,PARTNER_ZIP_CODE,PARTNER_CITY,PARTNER_ADRESS_COUNT,Sprachregion,Haushaltstyp,Urbanitaet,Wohneigentumverhaeltnis,Haushaltsgroesse,KaufkraftindexGemeinde,Fahrzeugtyp,Haushaltseinkommen,Fahrzeugklasse,Lebensphase,HaushaltsgroesseFein,Fahrzeugart,Altersklasse5J,Kanton,WEMFWirtschaftsgebiet,Altersklasse1J
0,13845507.0,2016-05-03 00:00:00.0000000,CHURN,1,0.0,98.40,6,2009-06-09 00:00:00.0000000,2014-07-03 00:00:00.0000000,52132.0,854.622951,10,83,NaN,24 Monate,21300.0,26 KMU Privatkunde,0,VW-PW,Polo Y&F 1.2 70PS 5G,9N31W4S5: Polo Young&Fresh 1.2 70 PS (2009),Benzin,Schaltgetriebe,Frontantrieb,5.0,5.0,Schwarz,Stoff Maxima anthrazit,9.0,POLIM,Polo,POLIM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13788514,NaN,1,Ostschweiz,"45 Motorfahrzeuge Handel, Instandhaltung & Rep...",Deutsch,1,0,1,1,NaN,Referenzadresse Postfach,NaN,SH,8201,Schaffhausen,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13845534.0,2015-09-13 00:00:00.0000000,CHURN,1,0.0,286.45,8,2009-02-23 00:00:00.0000000,2014-04-08 00:00:00.0000000,79790.0,1286.935484,7,79,SH,24 Monate,21300.0,26 KMU Privatkunde,0,VW-PW,Polo Y&F 1.2 70PS 5G,9N31W4S5: Polo Young&Fresh 1.2 70 PS (2009),Benzin,Schaltgetriebe,Frontantrieb,5.0,5.0,Schwarz,Stoff Maxima anthrazit,9.0,POLIM,Polo,POLIM,92032419.0,NaN,Deutsch,NaN,NaN,NaN,Felsenstieg 9,SH,8200,Schaffhausen,2.0,22.0,13788514,NaN,1,Ostschweiz,"45 Motorfahrzeuge Handel, Instandhaltung & Rep...",Deutsch,1,0,1,1,NaN,Referenzadresse Postfach,NaN,SH,8201,Schaffhausen,1,1,1,1,1,1.0,4.0,0,0.0,0,2,1.0,0.0,1.0,14.0,42.0,24.0
2,13845536.0,2016-08-16 00:00:00.0000000,CHURN,1,0.0,1771.45,8,2009-02-25 00:00:00.0000000,2015-06-17 00:00:00.0000000,70392.0,926.210526,9,90,ZH,12 Monate,21300.0,26 KMU Privatkunde,0,VW-PW,Polo Y&F 1.2 70PS 5G,9N31W4S5: Polo Young&Fresh 1.2 70 PS (2009),Benzin,Schaltgetriebe,Frontantrieb,5.0,5.0,Schwarz,Stoff Maxima anthrazit,9.0,POLIM,Polo,POLIM,128486660.0,NaN,Deutsch,NaN,NaN,NaN,Alte Brücke 8,ZH,8427,Rorbas,1.0,28.0,13788514,NaN,1,Ostschweiz,"45 Motorfahrzeuge Handel, Instandhaltung & Rep...",Deutsch,1,0,1,1,NaN,Referenzadresse Postfach,NaN,SH,8201,Schaffhausen,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13845540.0,2016-05-01 00:00:00.0000000,CHURN,1,0.0,0.00,5,2009-02-23 00:00:00.0000000,2014-07-01 00:00:00.0000000,52743.0,811.430769,13,87,NaN,24 Monate,21300.0,26 KMU Privatkunde,0,VW-PW,Polo Y&F 1.2 70PS 5G,9N31W4S5: Polo Young&Fresh 1.2 70 PS (2009),Benzin,Schaltgetriebe,Frontantrieb,5.0,5.0,Schwarz,Stoff Maxima anthrazit,9.0,POLIM,Polo,POLIM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13788514,NaN,1,Ostschweiz,"45 Motorfahrzeuge Handel, Instandhaltung & Rep...",Deutsch,1,0,1,1,NaN,Referenzadresse Postfach,NaN,SH,8201,Schaffhausen,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13845577.0,2015-11-24 00:00:00.0000000,CHURN,2,0.0,0.00,7,2009-12-23 00:00:00.0000000,2014-01-24 00:00:00.0000000,69832.0,1425.142857,7,71,JU,24 Monate,24380.0,26 KMU Privatkunde,0,VW-PW,POLO CL 1.4 85PS 5G,6R13E4CL: Polo Comfortline 1.4 16V 85 PS (2010),Benzin,Schaltgetriebe,Frontantrieb,5.0,5.0,Deepblack Perleffekt,Stoff Fonzie Bl

In [4]:
churnDataOrig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74130 entries, 0 to 74129
Data columns (total 77 columns):
car_nr                           74130 non-null float64
ref_dt                           74130 non-null object
target_event                     74130 non-null object
NUM_CONSEC_SERVICES              74130 non-null int64
SUM_INVOICE_AMOUNT_IN_SERVICE    74130 non-null float64
SUM_INVOICE_AMOUNT               74130 non-null float64
NUM_EVENTS                       74130 non-null int64
FIRST_EVT                        74130 non-null object
LAST_EVT                         74130 non-null object
LAST_MILEAGE                     74129 non-null float64
MEAN_MILEAGE_PER_MNTH            74130 non-null float64
AVG_DIFF_MNTH                    74130 non-null int64
age_mnth                         74130 non-null int64
KANTON_LICENCE_PLATE             68509 non-null object
INSPECTION_INTERVAL_UID          74130 non-null object
LIST_PRICE                       74129 non-null float64
END_CUSTO

### Create new distance variable (--> distance from customer ZIP-Code to partner address)
The distance between the approximate customer address (ZIP-Code only) and the exact partner address (=branch) will be queried from the GoogleMaps-API. First the partner addresses have to be updated with exacter data as there are a lot of postal adresses only in the main data set. (Note: calculating the distance with exact customer addresses has been considered, but would have been too expensive for this project.)

#### Prepare / get accurate partner adresses & geolocations (stored in separate file 'branches.csv')

(Note: For partners geolocations are available too. Might be nice for later visualizations.)

In [5]:
# create separate DataFrame 'partnerData' with one row of partner address data per branch
partnerData = churnDataOrig.drop_duplicates(subset='PARTNER_COMPANY_NR', keep='first')
partnerData = partnerData[['PARTNER_COMPANY_NR', 'PARTNER_STREET', 'PARTNER_ZIP_CODE', 'PARTNER_CITY']
                         ].sort_values(by = 'PARTNER_CITY')

In [6]:
# check
partnerData

,PARTNER_COMPANY_NR,PARTNER_STREET,PARTNER_ZIP_CODE,PARTNER_CITY
198,13794798,Rohrerstrasse 29,5000,Aarau
266,13792098,Bächliwis 29,8184,Bachenbülach
67,13790553,NaN,6501,Bellinzona
26,13790407,Wankdorffeldstrasse 60,3014,Bern
255,13794808,Via S. Carlo 6,6932,Breganzona
207,13795298,Via S. Carlo 4,6932,Breganzona
69,13788511,Römerstrasse 16,2555,Brügg
170,13788889,St. Gallerstrasse 100,9471,Buchs SG 1
6,13791153,Rue de Vuippens 55,1630,Bulle
169,13794801,Chemin de la Marbrerie 8,1227,Carouge


**Need for cleaning:**
- Eliminate dupliates in City column, as we will merge on that (too many differences in ZIP codes due to PO-Box addresses)
- Fill NaN in street column with street data from 'branches.csv'
- Update the ZIP codes with data from 'branches.csv'
- Add missing geolocations ( also from 'branches.csv')
- Create column with exact partner address

In [7]:
# eliminate duplicates in City column where different partners in one City
def funPartnerDuplicates(df):
    df.loc[(df['PARTNER_COMPANY_NR'] == 13790012), 'PARTNER_CITY'] = "Altstetten"
    df.loc[(df['PARTNER_COMPANY_NR'] == 13791152), 'PARTNER_CITY'] = "Zürich-Uto"
    df.loc[(df['PARTNER_COMPANY_NR'] == 13796110), 'PARTNER_CITY'] = "Zürich-Skoda"
    df.loc[(df['PARTNER_COMPANY_NR'] == 13792998), 'PARTNER_CITY'] = "Uster-NF"

funPartnerDuplicates(partnerData)
funPartnerDuplicates(churnDataOrig) # has to be performed on this DataFrame too, for later merging

# eliminate duplicates in City column where same partner has multiple addresses
partnerData = partnerData.drop_duplicates(subset='PARTNER_CITY', keep='first')

In [8]:
# check
partnerData.tail()

,PARTNER_COMPANY_NR,PARTNER_STREET,PARTNER_ZIP_CODE,PARTNER_CITY
30,13790796,Zürcherstrasse 312,8406,Winterthur
17,13789273,Gewerbestrasse 11,4528,Zuchwil
253,13790012,NaN,8048,Altstetten
42,13791152,NaN,8022,Zürich-Uto
563,13796110,Kalchbühlstrasse 44,8038,Zürich-Skoda


In [9]:
# read separate file 'branches' with better street data and geolocations
branches = pd.read_csv('branches.csv', sep=';', engine='python')
branches.columns

Index(['Nr.', 'Strasse', 'PLZ', 'Ort', 'Latitude', 'Longitude', 'VP VW',
       'VP Audi', 'VP SEAT', 'VP Skoda', 'VP NF', 'SP VW', 'SP Audi',
       'SP SEAT', 'SP Skoda', 'SP NF'],
      dtype='object')

In [10]:
# merge selected columns from 'branches' with 'partnerData'
partnerData = pd.merge(partnerData, branches[['Strasse', 'PLZ', 'Ort', 'Latitude', 'Longitude']], 
                    left_on='PARTNER_CITY', right_on= 'Ort', how='left')


In [11]:
# check
partnerData

,PARTNER_COMPANY_NR,PARTNER_STREET,PARTNER_ZIP_CODE,PARTNER_CITY,Strasse,PLZ,Ort,Latitude,Longitude
0,13794798,Rohrerstrasse 29,5000,Aarau,Rohrerstrasse 30,5000,Aarau,47.395074,8.057995
1,13792098,Bächliwis 29,8184,Bachenbülach,Bächliwis 29,8184,Bachenbülach,47.507737,8.543327
2,13790553,NaN,6501,Bellinzona,Via S. Gottardo 92,6500,Bellinzona,46.205429,9.035917
3,13790407,Wankdorffeldstrasse 60,3014,Bern,Wankdorffeldstrasse 60,3014,Bern,46.965961,7.465501
4,13794808,Via S. Carlo 6,6932,Breganzona,Via S. Carlo 6,6932,Breganzona,46.013807,8.932989
5,13788511,Römerstrasse 16,2555,Brügg,Römerstrasse 16,2555,Brügg,47.121787,7.261227
6,13788889,St. Gallerstrasse 100,9471,Buchs SG 1,St. Gallerstrasse 100,9470,Buchs SG 1,47.176572,9.467328
7,13791153,Rue de Vuippens 55,1630,Bulle,Route de Riaz 85,1630,Bulle,46.627984,7.060085
8,13794801,Chemin de la Marbrerie 8,1227,Carouge,Chemin de la Marbrerie 9,1227,Carouge,46.179119,6.135487
9,13795359,Alte Steinhauserstrasse 1,6330,Cham,Alte Steinhauserstrasse 1a,6330,Cham,47.186121,8.473649


In [12]:
# merge selcected columns into main dataframe
churnDataOrig = pd.merge(churnDataOrig, partnerData[['PARTNER_CITY', 'Latitude', 'Longitude', 'Strasse', 'PLZ']], how='left')
churnDataOrig = churnDataOrig.rename(columns={'Latitude': 'PARTNER_LAT', 'Longitude': 'PARTNER_LNG'})

# substitute values in column PARTNER_STREET / ZIP_CODE with more accurate data in columns Strasse, PLZ - drop those
churnDataOrig['PARTNER_STREET'] = churnDataOrig['Strasse']
churnDataOrig['PARTNER_ZIP_CODE'] = churnDataOrig['PLZ']
del churnDataOrig['Strasse']
del churnDataOrig['PLZ']

# create new column with complete customer address
churnDataOrig['PARTNER_ADDRESS'] = churnDataOrig['PARTNER_STREET']+', '+churnDataOrig[
    'PARTNER_ZIP_CODE'].astype(str)+' '+churnDataOrig['PARTNER_CITY']+', Switzerland'

In [13]:
#check
churnDataOrig.sample(40)

,car_nr,ref_dt,target_event,NUM_CONSEC_SERVICES,SUM_INVOICE_AMOUNT_IN_SERVICE,SUM_INVOICE_AMOUNT,NUM_EVENTS,FIRST_EVT,LAST_EVT,LAST_MILEAGE,MEAN_MILEAGE_PER_MNTH,AVG_DIFF_MNTH,age_mnth,KANTON_LICENCE_PLATE,INSPECTION_INTERVAL_UID,LIST_PRICE,END_CUSTOMER_STATUS_UID,ACCIDENT_CAR,CAR_BRAND_UID,DESCRIPTION_DE,DESCRIPTION_DE2,FUEL_TYPE_UID,GEAR_TYPE_UID,WHEEL_DRIVE_UID,NUMBER_OF_DOORS_UID,GEAR_COUNT,EXTERIOR_COLOR_DE,INTERIOR_COLOR_DE,BASE_MARGIN,BUDGET_TYPE_IMPORT,SALES_TYPE,SALES_TYPE_IMPORT,CAR_OWNER_JOIN_NR,FUNCTION_TYPE_UID,PERSON_LANGUAGE_UID,TITLE,POSITION,EVT_BIRTH,PERSON_STREET,PERSON_STATE,PERSON_ZIP_CODE,PERSON_CITY,PERSON_ADRESS_COUNT,ownerAge,PARTNER_COMPANY_NR,NAME,ACTIVE,REGION_UID,SECTOR_UID,PARTNER_LANGUAGE_UID,IS_RETAIL_PARTNER,IS_PREFERRED_PARTNER,IS_DEALER,IS_SERVICE_PARTNER,RATING_UID,PARTNER_ADRESS_TYPE_UID,PARTNER_STREET,PARTNER_STATE,PARTNER_ZIP_CODE,PARTNER_CITY,PARTNER_ADRESS_COUNT,Sprachregion,Haushaltstyp,Urbanitaet,Wohneigentumverhaeltnis,Haushaltsgroesse,KaufkraftindexGemeinde,Fahrzeugtyp,Haushaltseinkommen,Fahrzeugklasse,Lebensphase,HaushaltsgroesseFein,Fahrzeugart,Altersklasse5J,Kanton,WEMFWirtschaftsgebiet,Altersklasse1J,PARTNER_LAT,PARTNER_LNG,PARTNER_ADDRESS
31596,114333954.0,2016-12-03 00:00:00.0000000,ACTIVE,2,0.0,1740.10,21,2010-11-08 00:00:00.0000000,2016-11-22 00:00:00.0000000,62728.0,871.222222,3,73,BE,24 Monate,40480.0,03 Privatkunde,0,SKODA,OCTCOM ELE 1.4 122PS 7GDSG,1Z53LDEL: OCTAVIA COMBI ELEGANCE 1.4 TSI 122 P...,Benzin,Automat,Frontantrieb,5.0,7.0,magic schwarz perleffekt,schwarz/Armaturenbrett onyx/on,12.0,OCTCOM,Octavia Combi,OCTCOM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13790407,NaN,1,Espace Mittelland,"45 Motorfahrzeuge Handel, Instandhaltung & Rep...",Deutsch,1,1,1,1,NaN,Referenzadresse,Wankdorffeldstrasse 60,BE,3014,Bern,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.965961,7.465501,"Wankdorffeldstrasse 60, 3014 Bern, Switzerland"
49722,114425331.0,2015-08-15 00:00:00.0000000,CHURN,4,0.0,173.20,18,2009-10-21 00:00:00.0000000,2013-11-13 00:00:00.0000000,65527.0,1337.285714,2,70,TG,24 Monate,111305.0,03 Privatkunde,0,Audi,S4 AVANT 333 TFSI S-tronic,8K5S4Y: S4 Avant 3.0 TFSI 333 PS Direkt-Schalt...,Benzin,Automat,Allradantrieb,5.0,7.0,ibisweiss,schwarz / Himmel schwarz,7.0,A4AV,S4 Avant,S4AV,571472311.0,NaN,Deutsch,NaN,NaN,NaN,Löwenstrasse 7,SG,9500,Wil,2.0,NaN,13792098,NaN,1,Zürich,"45 Motorfahrzeuge Handel, Instandhaltung & Rep...",Deutsch,1,0,1,1,NaN,Firmenadresse,Bächliwis 29,ZH,8184,Bachenbülach,2,1,1,1,1,1.0,4.0,4,0.0,4,2,1.0,6.0,3.0,17.0,26.0,34.0,47.507737,8.543327,"Bächliwis 29, 8184 Bachenbülach, Switzerland"
67080,136920809.0,2015-12-10 00:00:00.0000000,ACTIVE,1,0.0,352.50,9,2013-04-11 00:00:00.0000000,2015-11-23 00:00:00.0000000,29272.0,944.258064,3,32,VD,24 Monate,32510.0,03 Privatkunde,0,VW-PW,Golf VII CL 1.4TSI 122PS DSG7,5G13GZCL: Golf VII Comfortline 1.4 TSI 122 PS ...,Benzin,Automat,Frontantrieb,5.0,7.0,Limestone Grey Metallic,Schwarz,10.0,GOLIM,Golf,GOLIM,207035673.0,NaN,Französisch,NaN,NaN,NaN,rue du Lac 4,VD,1815,Clarens,1.0,45.0,13795352,NaN,1,Genferseeregion,"45 Motorfahrzeuge Handel, Instandhaltung & Rep...",Französisch,1,0,1,1,NaN,Abweichende Postfachadresse,Avenue du Général-Guisan 78,VD,1800,Vevey 1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.466717,6.836412,"Avenue du Général-Guisan 78, 1800 Vevey 1, Swi..."
68582,144490683.0,2016-06-23 00:00:00.0000000,ACTIVE,2,0.0,710.95,12,2013-06-06 00:00:00.0000000,2016-04-22 00:00:00.0000000,77818.0,2288.764706,2,36,NaN,24 Monate,37400.0,03 Privatkunde,0,SKODA,NEW OCTLIM EL 2.0 150 PS 6G,5E3485EL: NEW OCTAVIA LIMOUSINE ELEGANCE 2.0 T...,Diesel,Schaltgetriebe,Frontantrieb,5.0,6.0,brilliant silber metallic,Satin schwarz,12.0,OCTLIM,Octavia Limousine,OCTLIM,206292826.0,Corporate,Französisch,NaN,NaN,NaN,rue du Midi 23,VD,1800,Vevey,3.0,34.0,13302288,NaN,1,Genferseeregion,"45 Motorfahrzeuge Handel, Instandhaltung & Rep...",Französisch,1,0,1,1,NaN,Firmenadresse,Route

#### Prepare origin & departure address pairs for GoogleMaps-API query

(Note to myself: I have queried with geolocations of partner sites, probably it would have been better with partner addresses, as I had to do some extra cleaning afterwards ...)

In [14]:
# create a list of tuples with unique combinations of origins (partner geocode) and departures (customer ZIP-Code, City)
pairs = set(zip((churnDataOrig['PARTNER_LAT'].astype(str)+", "+churnDataOrig['PARTNER_LNG'].astype(str)).tolist(), 
                (churnDataOrig['PERSON_ZIP_CODE'].astype(str)+", "+churnDataOrig['PERSON_CITY']).tolist()))

display("Number of unique pairs: "+str(len(pairs)))

'Number of unique pairs: 16203'

#### Request distances from GoogleMaps API and write to file

In [15]:
# gmaps = googlemaps.Client(key='')

# # download json-data and write to txt-file, line by line
# with open('distances.txt', 'a', encoding = 'utf-8') as f:
#     for origin, destination in tqdm_notebook(pairs): 
#         try:
#             distance = gmaps.distance_matrix(origin, destination, 
#                                              mode="driving",  
#                                              region=".ch")
#             json.dump(distance, f)
#             f.write("\n")
#         except Exception as e:
#             print(str(e))

#### Read and Merge distances with main dataframe

In [16]:
# read txt-File with Distance-Data
distances = pd.read_json('distances.txt', orient="records", lines=True)

In [17]:
# check
pd.set_option('max_colwidth', -1) 
display("Len of JSON data: "+str(len(distances)))
display(distances.sample(10))

'Len of JSON data: 29740'

,destination_addresses,origin_addresses,rows,status
11975,"[6262 Langnau bei Reiden, Switzerland]","[Sandblatte 3, 6026 Rain, Switzerland]","[{'elements': [{'distance': {'text': '27.3 km', 'value': 27326}, 'duration': {'text': '21 mins', 'value': 1245}, 'status': 'OK'}]}]",OK
20186,"[6026 Rain, Switzerland]","[Neuweidstrasse 1, 6208 Oberkirch, Switzerland]","[{'elements': [{'distance': {'text': '17.2 km', 'value': 17187}, 'duration': {'text': '17 mins', 'value': 1035}, 'status': 'OK'}]}]",OK
164,"[8600 Dübendorf, Switzerland]","[Oberfeldstrasse 24, 8302 Kloten, Switzerland]","[{'elements': [{'distance': {'text': '10.5 km', 'value': 10544}, 'duration': {'text': '15 mins', 'value': 898}, 'status': 'OK'}]}]",OK
29186,"[3186 Düdingen, Switzerland]","[Route de Villars 103, 1752 Villars-sur-Glâne, Switzerland]","[{'elements': [{'distance': {'text': '11.0 km', 'value': 10972}, 'duration': {'text': '11 mins', 'value': 666}, 'status': 'OK'}]}]",OK
3590,"[Schliern, 3098 Köniz, Switzerland]","[Worbstrasse 226, 3073 Gümligen, Switzerland]","[{'elements': [{'distance': {'text': '10.4 km', 'value': 10432}, 'duration': {'text': '23 mins', 'value': 1350}, 'status': 'OK'}]}]",OK
4647,"[8413 Neftenbach, Switzerland]","[Rautistrasse 22, 8047 Zürich, Switzerland]","[{'elements': [{'distance': {'text': '32.1 km', 'value': 32076}, 'duration': {'text': '36 mins', 'value': 2154}, 'status': 'OK'}]}]",OK
5027,"[3653 Oberhofen, Switzerland]","[Chemin de la Marbrerie 9, 1227 Carouge, Switzerland]","[{'elements': [{'distance': {'text': '205 km', 'value': 204753}, 'duration': {'text': '2 hours 21 mins', 'value': 8459}, 'status': 'OK'}]}]",OK
13058,"[6438 Ibach SZ, Switzerland]","[St. Gallerstrasse 101, 8645 Jona SG, Switzerland]","[{'elements': [{'distance': {'text': '38.6 km', 'value': 38563}, 'duration': {'text': '44 mins', 'value': 2651}, 'status': 'OK'}]}]",OK
22094,"[1299 Crans-près-Céligny, Switzerland]","[Ch. Jacques-Philibert-de-Sauvage 44, 1219 Châtelaine, Switzerland]","[{'elements': [{'distance': {'text': '28.6 km', 'value': 28611}, 'duration': {'text': '23 mins', 'value': 1403}, 'status': 'OK'}]}]",OK
3963,"[1530 Payerne, Switzerland]","[Grabenhofstrasse 10, 6010 Kriens, Switzerland]","[{'elements': [{'distance': {'text': '163 km', 'value': 163019}, 'duration': {'text': '1 hour 41 mins', 'value': 6085}, 'status': 'OK'}]}]",OK


**Need for cleaning:**

- Remove [ ] everywhere
- Extract customer ZIP-code and / or City (where ZIP-Code is missing ... unfortunately GM did not always return the ZIP-code)
- Extract partner ZIP-Code for merging with main dataframe
- Create columns with distance values (meters, seconds)
- Remove Rows with distance not found
- Fill-in missing ZIP-Codes with help of 'postcodesCH.csv'
- Drop duplicates (in the end)

In [18]:
distanceData = distances.copy()

# remove brackets
for col in distanceData.columns:
    distanceData[col] = distanceData[col].astype(str).str.replace('[','').str.replace(']','')
    distanceData[col] = distanceData[col].str.rstrip("'").str.lstrip("'")

# extract person_zip_code and only where missing also person place
distanceData['PARTNER_ZIP_CODE'] = distances['origin_addresses'].astype(str).str.extract('(\d{4,})')
distanceData['PERSON_ZIP_CODE'] = distances['destination_addresses'].astype(str).str.extract('(\d{4,})')
distanceData['PERSON_PLACE'] = np.where((distanceData['PERSON_ZIP_CODE'].str.contains('(\{d,})')), 
                                        distanceData['destination_addresses'].astype(str).str.split(', Switzerland').str.get(0),
                                        np.nan)

# extract dist_metres and dist_seconds from row column
distanceData['dist_metres'] = distanceData['rows'].astype(str).str.split('value').str.get(1).str.split('}').str.get(0)
distanceData['dist_metres'] = distanceData['dist_metres'].str.lstrip(" ': ").astype('float')
distanceData['dist_seconds'] = distanceData['rows'].astype(str).str.split('mins').str.get(1)
distanceData['dist_seconds'] = distanceData['dist_seconds'].str.extract('(\d+)').astype('float')

# extract distance status from row colum and remove all rows with status other than OK
distanceData['status2'] = distanceData['rows'].astype(str).str.split("status': ").str.get(1).str.rstrip('}}')
distanceData = distanceData.loc[(distanceData['status2'] == "'OK'")]

C:\Users\r2d4\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [19]:
# check
display(len(distanceData))
display(distanceData.sample(10))

29336

,destination_addresses,origin_addresses,rows,status,PARTNER_ZIP_CODE,PERSON_ZIP_CODE,PERSON_PLACE,dist_metres,dist_seconds,status2
17997,"5621 Zufikon, Switzerland","Giessenstrasse 4, 8600 Dübendorf, Switzerland","{'elements': {'distance': {'text': '31.0 km', 'value': 31014}, 'duration': {'text': '28 mins', 'value': 1668}, 'status': 'OK'}}",OK,8600,5621,NaN,31014.0,1668.0,'OK'
16574,"8352 Elsau, Switzerland","Giessenstrasse 4, 8600 Dübendorf, Switzerland","{'elements': {'distance': {'text': '23.7 km', 'value': 23701}, 'duration': {'text': '27 mins', 'value': 1635}, 'status': 'OK'}}",OK,8600,8352,NaN,23701.0,1635.0,'OK'
11824,"6315 Alosen, Switzerland","Zugerstrasse 9, 8810 Horgen, Switzerland","{'elements': {'distance': {'text': '27.2 km', 'value': 27159}, 'duration': {'text': '27 mins', 'value': 1640}, 'status': 'OK'}}",OK,8810,6315,NaN,27159.0,1640.0,'OK'
5468,"3123 Belp, Switzerland","Via Antonio Rinaldi 3, 6850 Mendrisio, Switzerland","{'elements': {'distance': {'text': '305 km', 'value': 305307}, 'duration': {'text': '3 hours 18 mins', 'value': 11853}, 'status': 'OK'}}",OK,6850,3123,NaN,305307.0,11853.0,'OK'
11660,"8185 Winkel, Switzerland","Utoquai 49, 8008 Zürich, Switzerland","{'elements': {'distance': {'text': '18.8 km', 'value': 18778}, 'duration': {'text': '28 mins', 'value': 1662}, 'status': 'OK'}}",OK,8008,8185,NaN,18778.0,1662.0,'OK'
11676,"Alpnach, Switzerland","Mooshüslistrasse 32, 6032 Emmen, Switzerland","{'elements': {'distance': {'text': '18.1 km', 'value': 18131}, 'duration': {'text': '18 mins', 'value': 1057}, 'status': 'OK'}}",OK,6032,NaN,Alpnach,18131.0,1057.0,'OK'
10902,"3366 Bettenhausen, Switzerland","Luzernstrasse 123, 4528 Zuchwil, Switzerland","{'elements': {'distance': {'text': '12.6 km', 'value': 12648}, 'duration': {'text': '17 mins', 'value': 1048}, 'status': 'OK'}}",OK,4528,3366,NaN,12648.0,1048.0,'OK'
1016,"8055 Zürich, Switzerland","Via S. Carlo 6, 6932 Breganzona, Switzerland","{'elements': {'distance': {'text': '205 km', 'value': 204657}, 'duration': {'text': '2 hours 29 mins', 'value': 8917}, 'status': 'OK'}}",OK,6932,8055,NaN,204657.0,8917.0,'OK'
9942,"3176 Neuenegg, Switzerland","Ch. Jacques-Philibert-de-Sauvage 44, 1219 Châtelaine, Switzerland","{'elements': {'distance': {'text': '158 km', 'value': 158111}, 'duration': {'text': '1 hour 39 mins', 'value': 5958}, 'status': 'OK'}}",OK,1219,3176,NaN,158111.0,5958.0,'OK'
1710,"7304 Maienfeld, Switzerland","Hauptstrasse 118, 8280 Kreuzlingen, Switzerland","{'elements': {'distance': {'text': '109 km', 'value': 109350}, 'duration': {'text': '1 hour 19 mins', 'value': 4765}, 'status': 'OK'}}",OK,8280,7304,NaN,109350.0,4765.0,'OK'


In [20]:
distData = distanceData.copy()

# drop columns not longer needed and drop duplicates on ZIP / PLACE combinations
distData.drop(['destination_addresses', 'origin_addresses', 'rows', 'status', 'status2'], axis=1, inplace=True)
distData = distData.drop_duplicates(subset=['PARTNER_ZIP_CODE', 'PERSON_ZIP_CODE', 'PERSON_PLACE'], keep='first')

In [21]:
# check
distData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15111 entries, 0 to 29739
Data columns (total 5 columns):
PARTNER_ZIP_CODE    15111 non-null object
PERSON_ZIP_CODE     13822 non-null object
PERSON_PLACE        1289 non-null object
dist_metres         15111 non-null float64
dist_seconds        14968 non-null float64
dtypes: float64(2), object(3)
memory usage: 708.3+ KB


In [22]:
# look at remaining entries without PERSON_ZIP_CODE
distData.loc[(distData['PERSON_PLACE'].notnull())].sort_values(by='PERSON_PLACE')

,PARTNER_ZIP_CODE,PERSON_ZIP_CODE,PERSON_PLACE,dist_metres,dist_seconds
22002,6517,NaN,"""Collina d'Oro",37296.0,2119.0
402,3280,NaN,"""L'Areuse",56818.0,3270.0
222,1752,NaN,"41-208 Sosnowiec, Poland",1318342.0,44505.0
17552,3014,NaN,"530 09 Pardubice, Czechia",931249.0,32287.0
17721,6512,NaN,Acquarossa,40120.0,1886.0
15861,8406,NaN,Adlikon bei Andelfingen,13639.0,691.0
24169,8600,NaN,Adlikon bei Andelfingen,28261.0,1309.0
24747,4528,NaN,Adlikon bei Andelfingen,122030.0,4782.0
614,8500,NaN,Adlikon bei Andelfingen,25277.0,1051.0
2324,8280,NaN,Affeltrangen,20795.0,1456.0


#### Replace (some) of the missing ZIP-codes 
(Note: unfortunatley GoogleMaps did not return ZIP-codes for some of the customer addresses, so that they cannot exactly be merged.)

In [23]:
# load new source (SwissPostCodes) 
postcodesCH = pd.read_csv('postcodesCH.csv', delimiter=";")
postcodesCH.drop_duplicates(subset='ORT', keep='first') # this means we won't get all NaNs
postcodesCH.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3495 entries, 0 to 3494
Data columns (total 5 columns):
PLZ          3495 non-null int64
ORT          3495 non-null object
Kanton       3495 non-null object
Latitude     3495 non-null float64
Longitude    3495 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 136.6+ KB


In [24]:
# merge dataframes and replace missing ZIP-codes where PERSON_PLACE was not NaN
distData = pd.merge(distData, postcodesCH[['ORT', 'PLZ']], left_on='PERSON_PLACE', right_on='ORT', how='left')
distData['PERSON_ZIP_CODE'] = np.where((distData['PERSON_PLACE'].notnull()), distData['PLZ'], distData['PERSON_ZIP_CODE'])
distData = distData.drop_duplicates(subset=['PARTNER_ZIP_CODE', 'PERSON_ZIP_CODE'], keep='first')

In [25]:
# check
distData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15790 entries, 0 to 16065
Data columns (total 7 columns):
PARTNER_ZIP_CODE    15790 non-null object
PERSON_ZIP_CODE     15733 non-null object
PERSON_PLACE        1968 non-null object
dist_metres         15790 non-null float64
dist_seconds        15648 non-null float64
ORT                 1911 non-null object
PLZ                 1911 non-null float64
dtypes: float64(3), object(4)
memory usage: 986.9+ KB


In [26]:
# look at remaining entries without PERSON_ZIP_CODE
distData.loc[(distData['PERSON_PLACE'].notnull()) & (distData['PERSON_ZIP_CODE'].isnull())].sort_values(by='PERSON_PLACE')

,PARTNER_ZIP_CODE,PERSON_ZIP_CODE,PERSON_PLACE,dist_metres,dist_seconds,ORT,PLZ
466,3280,NaN,"""L'Areuse",56818.0,3270.0,NaN,NaN
281,1752,NaN,"41-208 Sosnowiec, Poland",1318342.0,44505.0,NaN,NaN
678,8500,NaN,Adlikon bei Andelfingen,25277.0,1051.0,NaN,NaN
8110,6850,NaN,Alt St Johann,224927.0,9584.0,NaN,NaN
7970,8600,NaN,"Auvernier, Milvignes",162871.0,6851.0,NaN,NaN
2749,2023,NaN,Botterens,10491.0,722.0,NaN,NaN
10226,8570,NaN,Bussnang,5386.0,417.0,NaN,NaN
3703,7000,NaN,Cham,120540.0,4894.0,NaN,NaN
3124,8645,NaN,Cham,41021.0,2545.0,NaN,NaN
3120,5000,NaN,Cham,65420.0,2595.0,NaN,NaN


In [27]:
# drop all rows where PERSON_ZIP_CODE is still missing (they should not be merged in next step)
distData = distData[distData['PERSON_ZIP_CODE'].notnull()]
len(distData)

15733

In [28]:
churnData = churnDataOrig.copy()
churnData['PARTNER_ZIP_CODE'] = churnData['PARTNER_ZIP_CODE'].astype(str)
churnData = pd.merge(churnData, distData[['PARTNER_ZIP_CODE', 'PERSON_ZIP_CODE','dist_metres', 'dist_seconds']])

In [29]:
# check
churnData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56778 entries, 0 to 56777
Data columns (total 82 columns):
car_nr                           56778 non-null float64
ref_dt                           56778 non-null object
target_event                     56778 non-null object
NUM_CONSEC_SERVICES              56778 non-null int64
SUM_INVOICE_AMOUNT_IN_SERVICE    56778 non-null float64
SUM_INVOICE_AMOUNT               56778 non-null float64
NUM_EVENTS                       56778 non-null int64
FIRST_EVT                        56778 non-null object
LAST_EVT                         56778 non-null object
LAST_MILEAGE                     56777 non-null float64
MEAN_MILEAGE_PER_MNTH            56778 non-null float64
AVG_DIFF_MNTH                    56778 non-null int64
age_mnth                         56778 non-null int64
KANTON_LICENCE_PLATE             55490 non-null object
INSPECTION_INTERVAL_UID          56778 non-null object
LIST_PRICE                       56777 non-null float64
END_CUSTO

### Safe dataframe 'churnData' to csv

In [32]:
churnData.to_csv('churnDataWithDistances.csv', index=False)

### Conlusion
This operation did cost about 18'000 observations (about 25% of the total). If only the data where we had initial customer addresses is taken into account, the loss is about 10%. But it is assumed, that the distance from customer to partner is an important driver of loyality, and so this loss may be justified. In any case it will be interesting to check the correlation between our target variable and the distance in the next step.